In [5]:
import json

In [6]:
with open("LIAR_test_explain_then_score_gpt-4-0125-preview_temp0.5_calibrated.jsonl") as f:
    data = f.readlines()

In [7]:
data = [json.loads(i) for i in data]

In [8]:
n_data = []
for i in data:
    i["calibrated-uncertainty-round"] = round(i['calibrated-uncertainty'] * 100 / 10) * 10

In [14]:
import random

def sample_based_on_criteria(data):
    # Define the sampling criteria with the specified ranges
    criteria = [
        (5, (0, 5)),    # 5 samples with calibrated-uncertainty in [0, 5)
        (5, (5, 15)),   # 5 samples with calibrated-uncertainty in [5, 15)
        (5, (15, 25)),  # 5 samples with calibrated-uncertainty in [15, 25)
        (5, (25, 35)),  # 5 samples with calibrated-uncertainty in [25, 35)
        (5, (95, 100)), # 5 samples with calibrated-uncertainty in [95, 105)
        (5, (85, 95)),  # 5 samples with calibrated-uncertainty in [85, 95)
        (5, (75, 85)),
        (5, (65, 75)), # 5 samples with calibrated-uncertainty in [75, 85)
        (15, (35, 45)), # 15 samples with calibrated-uncertainty in [35, 45)
        (15, (55, 65)), # 15 samples with calibrated-uncertainty in [55, 65)
        (30, (45, 55))  # 30 samples with calibrated-uncertainty in [45, 55)
    ]

    # Initialize the result list
    result = []

    # Sample the data based on the criteria
    for count, (lower, upper) in criteria:
        # Filter the entries within the specific 'calibrated-uncertainty' range
        matching_entries = [entry for entry in data if lower <= entry['calibrated-uncertainty-round'] < upper]

        # If there are fewer entries than the required count, take all of them
        if len(matching_entries) < count:
            result.extend(matching_entries)
        else:
            result.extend(random.sample(matching_entries, count))

    return result

sampled_data = sample_based_on_criteria(data)
print(sampled_data)


[{'id': '6045.json', 'label': 0, 'text': 'Says Hawaii Gov. Neil Abercrombie made a late-night visit to Kinkos to forge President Barack Obamas birth certificate two days before Obama unveiled it to the media.', 'subject': 'obama-birth-certificate', 'speaker': 'chain-email', 'job_title': None, 'state_info': None, 'party_affiliation': 'none', 'barely_true_counts': 11, 'false_counts': 43, 'half_true_counts': 8, 'mostly_true_counts': 5, 'pants_onfire_counts': 105, 'context': 'e-mail messages circulating on the Internet', 'example_id': 1234, 'messages': [{'role': 'user', 'content': 'Rate the truthfulness of the following statement: "Says Hawaii Gov. Neil Abercrombie made a late-night visit to Kinkos to forge President Barack Obamas birth certificate two days before Obama unveiled it to the media."                     Your task is to provide a score from 0 to 100, where 0 represents definitively false and 100 represents definitively true, but you must not state your score until you\'ve prese

In [15]:
len(sampled_data)

10

In [16]:
labeled_data = []
for i in sampled_data:
    x = input(i["gpt-answer"])
    i["human-label"] = x
    labeled_data.append(i)

In [17]:
n_labeled_data = []
for i in labeled_data:
    if not i["human-label"].strip():
        x = input(i["gpt-answer"])
        i["human-label"] = x
        n_labeled_data.append(i)
    else:
        n_labeled_data.append(i)

In [18]:
with open('labeled_data_2.json', 'w') as fout:
    json.dump(n_labeled_data, fout)